# Siting Restaurants with MCLP in San Francisco

This notebook implements the maximize covering location problem (MCLP) to site 2 of 10 possible locations for restaurants in San Francisco's Mission District. The objective is to maximize the demand of randomly created households able to reach the restaurants within 3 minutes afoot. 

The routing matrix is computed using [Open Source Routing Machine's demo server API](http://project-osrm.org). 

In [209]:
import random
import json
import numpy as np
from shapely.geometry import shape, Point
import folium
from routingpy import OSRM
from spopt.locate.coverage import MCLP
import pulp

# San Francisco's Mission District Polygon
SF_MISSION_DISTRICT='{"type":"Polygon","coordinates":[[[-122.405115,37.764635],[-122.405212,37.763469],[-122.405832,37.762199],[-122.405859,37.761815],[-122.40604,37.761865],[-122.406365,37.761199],[-122.40645,37.760114],[-122.406167,37.759315],[-122.406015,37.759409],[-122.405604,37.758851],[-122.40405,37.757619],[-122.403428,37.756871],[-122.403585,37.756818],[-122.403328,37.756082],[-122.40315,37.754488],[-122.403524,37.754463],[-122.403437,37.753199],[-122.403364,37.752366],[-122.403022,37.752336],[-122.403052,37.752048],[-122.405091,37.745281],[-122.405614,37.7442],[-122.405493,37.744084],[-122.404865,37.744385],[-122.404524,37.744285],[-122.406652,37.741241],[-122.406936,37.740554],[-122.407045,37.739587],[-122.408136,37.73964],[-122.408009,37.737734],[-122.408218,37.73765],[-122.408284,37.736846],[-122.408613,37.736074],[-122.409287,37.735258],[-122.410052,37.734675],[-122.411978,37.733732],[-122.412695,37.733197],[-122.414554,37.732371],[-122.416255,37.732034],[-122.419881,37.732016],[-122.423718,37.73155],[-122.425944,37.731698],[-122.425256,37.732125],[-122.422274,37.732383],[-122.421889,37.732533],[-122.421787,37.732774],[-122.422472,37.733967],[-122.422901,37.734388],[-122.422243,37.734846],[-122.422133,37.735162],[-122.426983,37.735455],[-122.427849,37.734718],[-122.428578,37.735082],[-122.42801,37.735441],[-122.428454,37.735882],[-122.42551,37.737774],[-122.42452,37.739868],[-122.42427,37.739867],[-122.424394,37.740405],[-122.424168,37.740805],[-122.426453,37.764634],[-122.421885,37.764908],[-122.42173,37.763304],[-122.421031,37.76335],[-122.421248,37.764947],[-122.407553,37.765798],[-122.407431,37.764497],[-122.405115,37.764635]]]}'
# The amount of artificial households to be created
NUM_HOUSEHOLDS = 200

# This function generates a random point within a boundary
def generate_random(polygon):
    minx, miny, maxx, maxy = polygon.bounds
    return Point(random.uniform(minx, maxx), random.uniform(miny, maxy))

poly = shape(json.loads(SF_MISSION_DISTRICT))

# Until we have reached our maximal amount of households
households = []
while len(households) < NUM_HOUSEHOLDS:
    random_point = generate_random(poly)
    if (random_point.within(poly)):
        households.append([random_point.x, random_point.y])

    
    

## Adding a Leaflet Map for Visualisation via Folium

In the following we are using [folium](https://python-visualization.github.io/folium/) to visualize the data on the map. After creating the base map, we will load Mission District's boundary onto it and finally the artificial househoulds we created earlier. In the subsequent step we will add some fixed positions for our possible restaurants which are depicted as red circles.

In [210]:
m = folium.Map(location=[37.73522, -122.41065], tiles="cartodbpositron", zoom_start=13)

folium.GeoJson(data=SF_MISSION_DISTRICT, name="SF Mission District", style_function=lambda x: { "color": "orange", "fillColor": "orange"}).add_to(m)

households_fg = folium.FeatureGroup("Households").add_to(m)
for coord in households:
    folium.CircleMarker(
        location=[coord[1], coord[0]],
        radius=3,
        fill=True,
        fill_opacity=1,
        popup=folium.Popup("Random Household in Mission District", show=False),
        color="#3186cc"
    ).add_to(households_fg)

m


In [211]:
restaurants = [
    [-122.41957, 37.76032], 
    [-122.40789, 37.75781], 
    [-122.41525, 37.74622], 
    [-122.42265, 37.75189], 
    [-122.41890, 37.74224], 
    [-122.41365, 37.73596],
    [-122.40846, 37.76454],
    [-122.41489, 37.75905],
    [-122.42365, 37.74082],
    [-122.40667, 37.74899]
]

restaurants_fg = folium.FeatureGroup("Restaurants").add_to(m)
for coord in restaurants:
    folium.CircleMarker(
        location=[coord[1], coord[0]],
        radius=5,
        fill=True,
        fill_opacity=1,
        popup=folium.Popup("Potential Restaurants", show=False),
        color="red"
    ).add_to(restaurants_fg)

m

## Computing the Distance (Routing) Matrix

Given our 10 restaurants and 200 artificial households we want to compute our distance matrix. Instead of using simple euclidean distance we can make use of proper routes following roads which will yield more realistic travel times. The [routing-py library](https://github.com/gis-ops/routing-py) helps us achieve this which features the ability to use different open source frameworks such as OSRM or Valhalla as well as 3rd party API's such as Google Maps or HERE.

In [212]:
client = OSRM(base_url="https://router.project-osrm.org")
osrm_routing_matrix = client.matrix(
    dry_run=False,
    locations=restaurants + households, 
    sources=list(range(0,10)), 
    destinations=list(range(10, len(households) + 10)), 
    profile='pedestrian'
)
cost_matrix = np.array(osrm_routing_matrix.durations).T

## Solving the MCLP

Last but not least we want to run the solver. We choose the Pulp CBC solver and set a maximium coverage to 3 minutes. Our weights of the households we set to 1 and treat them equally. Finally we loop over the solution and add the sited restaurants and the corresponding allocated households to the map with different radiuses.  

In [216]:
solver = pulp.PULP_CBC_CMD(msg=False) # see solvers available in pulp reference
MAX_COVERAGE = 180 # maximum acceptable service duration, 3 minutes
ai = np.full((1, len(households)), 1, dtype=int) # weights, all set to 1 for demos sake

mclp_from_cost_matrix = MCLP.from_cost_matrix(cost_matrix, ai, MAX_COVERAGE, p_facilities=2)
result = mclp_from_cost_matrix.solve(solver)
mclp_from_cost_matrix.facility_client_array()

solution = {}
colors = ['orange', 'green', 'purple', 'cyan', 'magenta', 'orange', 'green', 'purple', 'cyan', 'magenta']

sited_restaurants_fg = folium.FeatureGroup("Sited Restaurants").add_to(m)
allocated_households_demand_fg = folium.FeatureGroup("Allocated Households").add_to(m)

for i in range(len(restaurants)):
    if mclp_from_cost_matrix.fac2cli[i]:
        folium.CircleMarker(
            location=[restaurants[i][1], restaurants[i][0]],
            radius=30,
            fill=True,
            popup=folium.Popup('Sited Restaurant', show=True),
            color=colors[i]
        ).add_to(sited_restaurants_fg)
        solution[i] = []
        for j in mclp_from_cost_matrix.fac2cli[i]:
            solution[i].append(households[j])
            folium.CircleMarker(
                location=[households[j][1], households[j][0]],
                radius=10,
                fill=True,
                popup="Household",
                color=colors[i]
            ).add_to(allocated_households_demand_fg)

folium.LayerControl().add_to(m)

m

In [187]:
result.uncovered_clients()
result.get_percentage()
result.percentage

solution


{4: [[-122.41993290803816, 37.74899423036618],
  [-122.41841812689285, 37.73542551027592],
  [-122.4246857987293, 37.7364040878501],
  [-122.4220550148617, 37.74656768125444],
  [-122.41540862169275, 37.74002228334895],
  [-122.41155931282132, 37.73949429697044],
  [-122.41949732067988, 37.744601519383615],
  [-122.42208755933294, 37.74413075722203],
  [-122.41817115541689, 37.74071636018459],
  [-122.42109111688744, 37.74551903411144],
  [-122.41626254162325, 37.7426450586693],
  [-122.41160869919706, 37.74067798721866],
  [-122.42151797714588, 37.73987451509924],
  [-122.41379920434495, 37.737630681318606],
  [-122.41986411638482, 37.740272503386244],
  [-122.42183123900706, 37.73549250560078],
  [-122.42105921326258, 37.74683710248359],
  [-122.42064833635979, 37.75010328369821],
  [-122.4150246884688, 37.745268387227874],
  [-122.42343610838495, 37.74749569609016],
  [-122.41606166347867, 37.73399872734114],
  [-122.41121649025334, 37.74061740995166],
  [-122.42018266908607, 37.738